In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import string
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
import tensorflow as tf

In [2]:
dfraw=pd.read_csv('ghc_test.tsv',sep='\t')

In [3]:
dfraw

,text,hd,cv,vo
0,https://www.youtube.com/watch?v=kACWpKAKtak A ...,0,0,0
1,Very nice! I tend to get tired of the constant...,0,0,0
2,Watch today. https://circumcisionmovie.com/,0,0,0
3,""" Thinking Venues "" First Color Layer blocking...",0,0,0
4,What about death penalty for perpetrators and...,0,0,0
...,...,...,...,...
5505,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,...",0,0,0
5506,i guess eu is gonna have to back track a littl...,0,0,0
5507,A good read here.... https://countrysquire....,0,0,0
5508,The only way to change things is to have compa...,0,0,0


In [4]:
dfraw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5510 entries, 0 to 5509
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5510 non-null   object
 1   hd      5510 non-null   int64 
 2   cv      5510 non-null   int64 
 3   vo      5510 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 172.3+ KB


In [5]:
dfraw['label']=dfraw['hd']+dfraw['cv']+dfraw['vo']

In [6]:
dfraw.head()

,text,hd,cv,vo,label
0,https://www.youtube.com/watch?v=kACWpKAKtak A ...,0,0,0,0
1,Very nice! I tend to get tired of the constant...,0,0,0,0
2,Watch today. https://circumcisionmovie.com/,0,0,0,0
3,""" Thinking Venues "" First Color Layer blocking...",0,0,0,0
4,What about death penalty for perpetrators and...,0,0,0,0


In [7]:
dfraw.isna().sum()

text     0
hd       0
cv       0
vo       0
label    0
dtype: int64

In [8]:
dfraw['label'].unique()

array([0, 1, 2, 3])

In [9]:
dfraw=dfraw.replace({'label':{3:2}})

In [10]:
dfraw['label'].unique()

array([0, 1, 2])

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
from nltk.corpus import stopwords

In [13]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [14]:
import re
dfraw['text'] = dfraw['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [15]:
dfraw

,text,hd,cv,vo,label
0,,0,0,0,0
1,Very nice! I tend to get tired of the constant...,0,0,0,0
2,Watch today.,0,0,0,0
3,""" Thinking Venues "" First Color Layer blocking...",0,0,0,0
4,What about death penalty for perpetrators and...,0,0,0,0
...,...,...,...,...,...
5505,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,...",0,0,0,0
5506,i guess eu is gonna have to back track a littl...,0,0,0,0
5507,A good read here....,0,0,0,0
5508,The only way to change things is to have compa...,0,0,0,0


In [16]:
dfraw.isna().sum()

text     0
hd       0
cv       0
vo       0
label    0
dtype: int64

In [18]:
dfraw1=dfraw.drop(labels=0, axis=0)
dfraw1

,text,hd,cv,vo,label
1,Very nice! I tend to get tired of the constant...,0,0,0,0
2,Watch today.,0,0,0,0
3,""" Thinking Venues "" First Color Layer blocking...",0,0,0,0
4,What about death penalty for perpetrators and...,0,0,0,0
5,You have insecurities? Fuck you,0,0,0,0
...,...,...,...,...,...
5505,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,...",0,0,0,0
5506,i guess eu is gonna have to back track a littl...,0,0,0,0
5507,A good read here....,0,0,0,0
5508,The only way to change things is to have compa...,0,0,0,0


In [20]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

# Remove Accented Chars
def remove_accented_chars_func(text):
    '''
    Removes all accented characters from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without accented characters
    '''
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
# Remove punctuation
def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)
# Remove Irrelevant Char
def remove_irr_char_func(text):
    '''
    Removes all irrelevant characters (numbers and punctuation) from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without irrelevant characters
    '''
    return re.sub(r'[^a-zA-Z]', ' ', text)
#Remove Whitespaces
def remove_extra_whitespaces_func(text):
    '''
    Removes extra whitespaces from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without extra whitespaces
    ''' 
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()
#lemmatization
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    text=remove_accented_chars_func(text)
    text=remove_punctuation_func(text)
    text=remove_irr_char_func(text)
    text=remove_extra_whitespaces_func(text)
    return text
#Apply function on review column
dfraw1['text']=dfraw1['text'].apply(denoise_text)

dfraw1

,text,hd,cv,vo,label
1,nice tend get tired constant stream ridiculous...,0,0,0,0
2,Watch today,0,0,0,0
3,Thinking Venues First Color Layer blocking Fig...,0,0,0,0
4,death penalty perpetrators expelling remaining...,0,0,0,0
5,insecurities Fuck,0,0,0,0
...,...,...,...,...,...
5505,Trump Counter DNC Lawsuit Seeks Servers Clinto...,0,0,0,0
5506,guess eu gonna back track little usa says matt...,0,0,0,0
5507,good read here,0,0,0,0
5508,way change things compassion rational possible...,0,0,0,0


In [21]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [22]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
  words = text.split()
  words = [lemmatizer.lemmatize(word,pos='v') for word in words]
  return ' '.join(words)
dfraw1['text'] = dfraw1['text'].apply(lemmatize_words)

dfraw1

[nltk_data] Downloading package wordnet to /root/nltk_data...


,text,hd,cv,vo,label
1,nice tend get tire constant stream ridiculousl...,0,0,0,0
2,Watch today,0,0,0,0
3,Thinking Venues First Color Layer block Figure...,0,0,0,0
4,death penalty perpetrators expel remain rest f...,0,0,0,0
5,insecurities Fuck,0,0,0,0
...,...,...,...,...,...
5505,Trump Counter DNC Lawsuit Seeks Servers Clinto...,0,0,0,0
5506,guess eu gonna back track little usa say matte...,0,0,0,0
5507,good read here,0,0,0,0
5508,way change things compassion rational possible...,0,0,0,0


In [23]:
dfclean01=dfraw1[['text', 'label']]
dfclean01

,text,label
1,nice tend get tire constant stream ridiculousl...,0
2,Watch today,0
3,Thinking Venues First Color Layer block Figure...,0
4,death penalty perpetrators expel remain rest f...,0
5,insecurities Fuck,0
...,...,...
5505,Trump Counter DNC Lawsuit Seeks Servers Clinto...,0
5506,guess eu gonna back track little usa say matte...,0
5507,good read here,0
5508,way change things compassion rational possible...,0


In [25]:
dfclean01['Corpus Name']='The GabHate corpus'
dfclean01

<ipython-input-25-471f46c06f50>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfclean01['Corpus Name']='The GabHate corpus'


,text,label,Corpus Name
1,nice tend get tire constant stream ridiculousl...,0,The GabHate corpus
2,Watch today,0,The GabHate corpus
3,Thinking Venues First Color Layer block Figure...,0,The GabHate corpus
4,death penalty perpetrators expel remain rest f...,0,The GabHate corpus
5,insecurities Fuck,0,The GabHate corpus
...,...,...,...
5505,Trump Counter DNC Lawsuit Seeks Servers Clinto...,0,The GabHate corpus
5506,guess eu gonna back track little usa say matte...,0,The GabHate corpus
5507,good read here,0,The GabHate corpus
5508,way change things compassion rational possible...,0,The GabHate corpus


In [27]:
dfclean01['raw_sentence']=dfraw['text']
dfclean01

<ipython-input-27-a9e5f15552cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfclean01['raw_sentence']=dfraw['text']


,text,label,Corpus Name,raw_sentence
1,nice tend get tire constant stream ridiculousl...,0,The GabHate corpus,Very nice! I tend to get tired of the constant...
2,Watch today,0,The GabHate corpus,Watch today.
3,Thinking Venues First Color Layer block Figure...,0,The GabHate corpus,""" Thinking Venues "" First Color Layer blocking..."
4,death penalty perpetrators expel remain rest f...,0,The GabHate corpus,What about death penalty for perpetrators and...
5,insecurities Fuck,0,The GabHate corpus,You have insecurities? Fuck you
...,...,...,...,...
5505,Trump Counter DNC Lawsuit Seeks Servers Clinto...,0,The GabHate corpus,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,..."
5506,guess eu gonna back track little usa say matte...,0,The GabHate corpus,i guess eu is gonna have to back track a littl...
5507,good read here,0,The GabHate corpus,A good read here....
5508,way change things compassion rational possible...,0,The GabHate corpus,The only way to change things is to have compa...


In [28]:
dfclean02=dfclean01[['Corpus Name','raw_sentence','label']]
dfclean02

,Corpus Name,raw_sentence,label
1,The GabHate corpus,Very nice! I tend to get tired of the constant...,0
2,The GabHate corpus,Watch today.,0
3,The GabHate corpus,""" Thinking Venues "" First Color Layer blocking...",0
4,The GabHate corpus,What about death penalty for perpetrators and...,0
5,The GabHate corpus,You have insecurities? Fuck you,0
...,...,...,...
5505,The GabHate corpus,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,...",0
5506,The GabHate corpus,i guess eu is gonna have to back track a littl...,0
5507,The GabHate corpus,A good read here....,0
5508,The GabHate corpus,The only way to change things is to have compa...,0


In [29]:
dfclean02['clean_sentence_training']=dfclean01['text']
dfclean02

,Corpus Name,raw_sentence,label,clean_sentence_training
1,The GabHate corpus,Very nice! I tend to get tired of the constant...,0,nice tend get tire constant stream ridiculousl...
2,The GabHate corpus,Watch today.,0,Watch today
3,The GabHate corpus,""" Thinking Venues "" First Color Layer blocking...",0,Thinking Venues First Color Layer block Figure...
4,The GabHate corpus,What about death penalty for perpetrators and...,0,death penalty perpetrators expel remain rest f...
5,The GabHate corpus,You have insecurities? Fuck you,0,insecurities Fuck
...,...,...,...,...
5505,The GabHate corpus,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,...",0,Trump Counter DNC Lawsuit Seeks Servers Clinto...
5506,The GabHate corpus,i guess eu is gonna have to back track a littl...,0,guess eu gonna back track little usa say matte...
5507,The GabHate corpus,A good read here....,0,good read here
5508,The GabHate corpus,The only way to change things is to have compa...,0,way change things compassion rational possible...


In [30]:
dfclean02['clean_sentence_EDA']=dfclean01['text']
dfclean02

,Corpus Name,raw_sentence,label,clean_sentence_training,clean_sentence_EDA
1,The GabHate corpus,Very nice! I tend to get tired of the constant...,0,nice tend get tire constant stream ridiculousl...,nice tend get tire constant stream ridiculousl...
2,The GabHate corpus,Watch today.,0,Watch today,Watch today
3,The GabHate corpus,""" Thinking Venues "" First Color Layer blocking...",0,Thinking Venues First Color Layer block Figure...,Thinking Venues First Color Layer block Figure...
4,The GabHate corpus,What about death penalty for perpetrators and...,0,death penalty perpetrators expel remain rest f...,death penalty perpetrators expel remain rest f...
5,The GabHate corpus,You have insecurities? Fuck you,0,insecurities Fuck,insecurities Fuck
...,...,...,...,...,...
5505,The GabHate corpus,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,...",0,Trump Counter DNC Lawsuit Seeks Servers Clinto...,Trump Counter DNC Lawsuit Seeks Servers Clinto...
5506,The GabHate corpus,i guess eu is gonna have to back track a littl...,0,guess eu gonna back track little usa say matte...,guess eu gonna back track little usa say matte...
5507,The GabHate corpus,A good read here....,0,good read here,good read here
5508,The GabHate corpus,The only way to change things is to have compa...,0,way change things compassion rational possible...,way change things compassion rational possible...


In [31]:
dfclean02.to_csv('ghc_clean02.csv')